
# Estudo do Problema da Dieta

### Autor: José Brito

### Introdução

Neste estudo, vamos explorar a formulação de um problema de dieta utilizando AMPL. Nosso objetivo é determinar a quantidade ótima de alimentos a serem consumidos para minimizar o custo total, atendendo às restrições nutricionais fornecidas. O problema será dividido em duas partes: o modelo (`diet.mod`) e os dados (`diet.dat`).

### Tabela de Alimentos

A tabela abaixo descreve os alimentos que serão considerados no modelo, juntamente com suas propriedades nutricionais e preços.

A tabela foi tirada do dataset disponível em: [dataset no Kaggle](https://www.kaggle.com/datasets/omsaraksa/diet-for-linear-programing2020?select=Diet_KSA_2020+%281%29.csv)

| Index | Nome            | Máx. Porções | Tamanho (g) | Energia (cal) | Proteína (g) | Cálcio (mg) | Magnésio (mg) | Vitamina C (mg) | Ferro (mg) | Preço (R$)  |
|-------|-----------------|--------------|-------------|---------------|--------------|-------------|---------------|-----------------|------------|-------------|
| 0     | Oatmeal         | 4            | 28          | 110           | 4            | 2           | 26            | 0               | 6          | 10.57       |
| 1     | Chicken         | 3            | 100         | 205           | 32           | 12          | 1.2           | 2               | 1.3        | 16.25       |
| 2     | Eggs            | 2            | 2           | 160           | 13           | 54          | 10            | 0               | 1.2        | 1.6         |
| 3     | Whole Milk      | 8            | 237         | 160           | 8            | 285         | 0             | 0               | 11         | 8           |
| 4     | Cherry pie      | 2            | 170         | 420           | 4            | 22          | 9             | 1               | 1.9        | 25          |
| 5     | Pork with beans | 2            | 206         | 260           | 14           | 80          | 31            | 0.6             | 0.8        | 4           |
| 6     | Yogurt          | 3            | 3           | 58            | 10           | 110         | 11            | 0.1             | 0.1        | 10          |
| 7     | Rice            | 5            | 30          | 500           | 9.4          | 28          | 25            | 0.01            | 0.8        | 8.9         |
| 8     | Meat            | 6            | 143         | 143           | 26           | 6           | 29            | 0               | 1.2        | 40          |
| 9     | Potato          | 5            | 54          | 323           | 2            | 12          | 28            | 19              | 0.78       | 3.29        |


In [ ]:
# Instalando as dependências
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["coin"],
    license_uuid="default",
)

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


### Explicação do Modelo

O objetivo é minimizar o custo total dos alimentos, garantindo que as necessidades nutricionais sejam atendidas. A função objetivo é dada por:

$$
\text{Minimizar custo_total} = \sum_{j \in \text{alimento}} \text{preco}[j] \times \text{Compra}[j]
$$

Onde:
- **preco[j]** é o preço do alimento \(j\).
- **Compra[j]** é a quantidade do alimento \(j\) a ser comprada.

As restrições garantem que a ingestão total de cada nutriente esteja dentro dos limites especificados:

$$
\text{n_min}[i] \leq \sum_{j \in \text{alimento}} \text{nutritente}[i][j] \times \text{Compra}[j] \leq \text{n_max}[i]
$$

Para cada nutriente \(i\) (energia, proteína, cálcio, magnésio, vitamina C), onde:
- **nutriente[i][j]** é a quantidade do nutriente \(i\) no alimento \(j\).
- **n\_min[i]** é o limite mínimo para o nutriente \(i\).
- **n\_max[i]** é o limite máximo para o nutriente \(i\).

In [ ]:
%%writefile dieta.mod
# Definição do conjunto de alimentos
set ALIMENTO;

# Parâmetros dos alimentos
param max_porcoes{ALIMENTO};  # Quantidade máxima permitida de cada alimento
param tamanho{ALIMENTO};      # Tamanho da porção (em gramas ou mL)
param energia{ALIMENTO};      # Energia (kcal)
param proteina{ALIMENTO};     # Proteína (g)
param calcio{ALIMENTO};       # Cálcio (mg)
param magnesio{ALIMENTO};     # Magnésio (mg)
param vitaminaC{ALIMENTO};    # Vitamina C (mg)
param ferro{ALIMENTO};        # Ferro (mg)
param preco{ALIMENTO};        # Preço (unidade monetária)

# Limites mínimos e máximos para nutrientes
param n_min{1..5};  # Valores mínimos recomendados
param n_max{1..5};  # Valores máximos permitidos

# Variáveis de decisão
var Compra{j in ALIMENTO} integer >= 0, <= max_porcoes[j];

# Função objetivo: minimizar o custo total da dieta
minimize Custo_Total: sum{j in ALIMENTO} preco[j] * Compra[j];

# Restrições de nutrientes
subject to Energia:   n_min[1] <= sum{j in ALIMENTO} energia[j] * Compra[j]   <= n_max[1];
subject to Proteina:  n_min[2] <= sum{j in ALIMENTO} proteina[j] * Compra[j]  <= n_max[2];
subject to Calcio:    n_min[3] <= sum{j in ALIMENTO} calcio[j] * Compra[j]    <= n_max[3];
subject to Magnesio:  n_min[4] <= sum{j in ALIMENTO} magnesio[j] * Compra[j]  <= n_max[4];
subject to VitaminaC: n_min[5] <= sum{j in ALIMENTO} vitaminaC[j] * Compra[j] <= n_max[5];


Overwriting dieta.mod


In [ ]:
%%writefile dieta.dat
# Definição do conjunto de alimentos
set ALIMENTO := Aveia Frango Ovos Leite_Integral Torta_de_Cereja Feijao_com_Carne Iogurte Arroz Carne Batata;

# Parâmetros dos alimentos
param: max_porcoes tamanho energia proteina calcio magnesio vitaminaC ferro preco :=
Aveia               4 28 110  4  2 26  0 6 10.57
Frango              3 100 205 32 12 1.2 2 1.3 16.25
Ovos                2 2 160 13 54 10  0 1.2 1.6
Leite_Integral      8 237 160 8 285 0  0 11 8
Torta_de_Cereja     2 170 420 4  22 9  1 1.9 25
Feijao_com_Carne    2 206 260 14 80 31  0.6 0.8 4
Iogurte             3 3 58 10 110 11  0.1 0.1 10
Arroz               5 30 500 9.4 28 25  0.01 0.8 8.9
Carne               6 143 143 26 6 29  0 1.2 40
Batata              5 54 323 2  12 28  19 0.78 3.29;

# Limites mínimos e máximos para nutrientes
param n_min :=
1 2500  # Energia mínima (kcal) - baseado na média para um homem adulto ativo
2 56    # Proteína mínima (g) - recomendação padrão para um homem adulto
3 1000  # Cálcio mínimo (mg) - essencial para a saúde óssea
4 400   # Magnésio mínimo (mg) - para funções musculares e nervosas
5 90;   # Vitamina C mínima (mg) - para imunidade e saúde celular

param n_max :=
1 3000  # Energia máxima (kcal) - valor superior para alta atividade física
2 100   # Proteína máxima (g) - limite seguro para consumo elevado
3 1300  # Cálcio máximo (mg) - acima disso pode aumentar risco de cálculos renais
4 420   # Magnésio máximo (mg) - baseado na ingestão diária recomendada
5 200;  # Vitamina C máxima (mg) - acima disso pode causar efeitos colaterais leves


Overwriting dieta.dat


In [ ]:
%%ampl_eval
reset;
model dieta.mod;
data dieta.dat;
option solver cbc;
solve;
display Compra;

cbc 2.10.12: cbc 2.10.12: infeasible problem
0 simplex iterations
absmipgap=1.79769e+308, relmipgap=1
Compra [*] :=
           Arroz    1.65177
           Aveia    4
          Batata    5
           Carne    4.45169
Feijao_com_Carne    2
          Frango    0
         Iogurte    3
  Leite_Integral    3.36759
            Ovos  -10.9393
 Torta_de_Cereja    0
;



O erro ocorreu porque não foi possível encontrar uma solução viável com os parâmetros atuais, resultando na exibição dos últimos valores testados (algo parecido ocorreu no documento que foi alvo do estudo original).

Agora, tomaremos as medidas necessárias para ajustar as restrições e os parâmetros, visando encontrar uma solução viável e otimizada.

As restrições de nutrientes podem estar muito apertadas, o que impede a obtenção de uma solução viável. Por exemplo, o valor de ```n_min``` para energia está muito próximo do valor de ```n_max``` em alguns casos. Pode ser que o modelo esteja tentando consumir alimentos de forma tão específica que é impossível atender a todas as restrições simultaneamente.

Agora, vou relaxar um pouco as restrições de nutrientes, aumentando alguns valores de ```n_max``` e diminuindo alguns valores de ```n_min``` para garantir mais flexibilidade.

In [ ]:
%%writefile dieta1.dat
# Definição do conjunto de alimentos
set ALIMENTO := Aveia Frango Ovos Leite_Integral Torta_de_Cereja Feijao_com_Carne Iogurte Arroz Carne Batata;

# Parâmetros dos alimentos
param: max_porcoes tamanho energia proteina calcio magnesio vitaminaC ferro preco :=
Aveia               4 28 110  4  2 26  0 6 10.57
Frango              5 100 205 32 12 1.2 2 1.3 16.25 # Aumentei a quantidade máxima para Frango
Ovos                3 2 160 13 54 10  0 1.2 1.6 # Aumentei a quantidade máxima para Ovos
Leite_Integral      8 237 160 8 285 0  0 11 8
Torta_de_Cereja     2 170 420 4  22 9  1 1.9 25
Feijao_com_Carne    3 206 260 14 80 31  0.6 0.8 4 # Aumentei a quantidade máxima para Feijão com Carne
Iogurte             4 3 58 10 110 11  0.1 0.1 10 # Aumentei a quantidade máxima para Iogurte
Arroz               6 30 500 9.4 28 25  0.01 0.8 8.9 # Aumentei a quantidade máxima para Arroz
Carne               6 143 143 26 6 29  0 1.2 40
Batata              6 54 323 2  12 28  19 0.78 3.29; # Aumentei a quantidade máxima para Batata

# Limites mínimos e máximos para nutrientes
param n_min :=
1 2200  # Energia mínima (kcal) - reduzi um pouco a exigência de energia
2 50    # Proteína mínima (g) - diminui um pouco a quantidade de proteína
3 800   # Cálcio mínimo (mg) - reduzi um pouco o mínimo de cálcio
4 300   # Magnésio mínimo (mg) - reduzi o mínimo de magnésio
5 70;   # Vitamina C mínima (mg) - reduzi o mínimo de vitamina C

param n_max :=
1 3500  # Energia máxima (kcal) - aumentei a máxima de energia
2 200   # Proteína máxima (g) - aumentei o máximo de proteína
3 1400  # Cálcio máximo (mg) - aumentei o máximo de cálcio
4 500   # Magnésio máximo (mg) - aumentei o máximo de magnésio
5 220;  # Vitamina C máxima (mg) - aumentei o máximo de vitamina C


Overwriting dieta1.dat


In [ ]:
%%ampl_eval
reset;
model dieta.mod;
data dieta1.dat;
option solver cbc;
solve;
display Compra;

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 54.54
0 simplex iterations
Compra [*] :=
           Arroz  0
           Aveia  0
          Batata  6
           Carne  0
Feijao_com_Carne  3
          Frango  0
         Iogurte  1
  Leite_Integral  1
            Ovos  3
 Torta_de_Cereja  0
;



### *Análise da solução ótima*

O modelo agora encontrou uma solução ótima com um custo total de $54.54. Os valores de compra para os alimentos são os seguintes:

- Arroz: 0 porções
- Aveia: 0 porção
- Batata: 6 porções
- Carne: 0 porções
- Feijão com Carne: 3 porções
- Frango: 0 porções
- Iogurte: 1 porções
- Leite Integral: 1 porção
- Ovos: 3 porções
- Torta de Cereja: 0 porções

Essa solução parece viável e otimizada, pois respeita as restrições de nutrientes e minimiza o custo total da dieta.

Embora a solução encontrada seja ótima em termos de custo, ela não é muito diversificada, pois seleciona poucos alimentos (Batata, Feijão com Carne, Iogurte, Leite Integral e Ovos). Isso pode ser um problema dependendo do objetivo da dieta, já que uma alimentação equilibrada e variada é importante para fornecer uma gama completa de nutrientes.

Para aumentar a diversidade de alimentos, fiz duas alterações:
- Modificação da função objetivo para incluir uma penalização pela falta de variedade:
```
# Função objetivo: minimizar o custo total da dieta e penalizar a falta de diversidade
minimize Custo_Total:
    sum{j in ALIMENTO} preco[j] * Compra[j] + 0.5 * (card(ALIMENTO) - sum{j in ALIMENTO} (Compra[j] > 0));
```
- Limitar a quantidade máxima de porções de alimentos repetidos:
```
# Limitação da quantidade máxima de porções de alimentos individuais
subject to Max_Porcao_Individual {j in ALIMENTO}:
    Compra[j] <= 4;  # Limitando a no máximo 4 porções por alimento

```

In [ ]:
%%writefile dieta1.mod
# Definição do conjunto de alimentos
set ALIMENTO;

# Parâmetros dos alimentos
param max_porcoes{ALIMENTO};  # Quantidade máxima permitida de cada alimento
param tamanho{ALIMENTO};      # Tamanho da porção (em gramas ou mL)
param energia{ALIMENTO};      # Energia (kcal)
param proteina{ALIMENTO};     # Proteína (g)
param calcio{ALIMENTO};       # Cálcio (mg)
param magnesio{ALIMENTO};     # Magnésio (mg)
param vitaminaC{ALIMENTO};    # Vitamina C (mg)
param ferro{ALIMENTO};        # Ferro (mg)
param preco{ALIMENTO};        # Preço (unidade monetária)

# Limites mínimos e máximos para nutrientes
param n_min{1..5};  # Valores mínimos recomendados
param n_max{1..5};  # Valores máximos permitidos

# Variáveis de decisão
var Compra{j in ALIMENTO} integer >= 0, <= max_porcoes[j];

# Variável auxiliar para indicar se um alimento foi escolhido (1 se foi escolhido, 0 caso contrário)
var Escolhido{j in ALIMENTO} binary;

# Restrição para ativar Escolhido apenas se Compra[j] for maior que zero
subject to Ativa_Escolhido {j in ALIMENTO}:
    Compra[j] <= max_porcoes[j] * Escolhido[j];

# Função objetivo: minimizar o custo total da dieta, incentivando a diversidade
minimize Custo_Total:
    sum{j in ALIMENTO} preco[j] * Compra[j]
    + 0.5 * (card(ALIMENTO) - sum{j in ALIMENTO} Escolhido[j]);  # Penaliza poucas escolhas

# Restrições de nutrientes
subject to Energia:   n_min[1] <= sum{j in ALIMENTO} energia[j] * Compra[j]   <= n_max[1];
subject to Proteina:  n_min[2] <= sum{j in ALIMENTO} proteina[j] * Compra[j]  <= n_max[2];
subject to Calcio:    n_min[3] <= sum{j in ALIMENTO} calcio[j] * Compra[j]    <= n_max[3];
subject to Magnesio:  n_min[4] <= sum{j in ALIMENTO} magnesio[j] * Compra[j]  <= n_max[4];
subject to VitaminaC: n_min[5] <= sum{j in ALIMENTO} vitaminaC[j] * Compra[j] <= n_max[5];

# Restrição para evitar que um único alimento seja dominante (máximo de 4 porções por alimento)
subject to Limite_Individual {j in ALIMENTO}:
    Compra[j] <= 4;


Overwriting dieta1.mod


In [ ]:
%%ampl_eval
reset;
model dieta1.mod;
data dieta1.dat;
option solver cbc;
solve;
display Compra;

cbc 2.10.12: cbc 2.10.12: optimal solution; objective 74.4
323 simplex iterations
323 barrier iterations
Compra [*] :=
           Arroz  1
           Aveia  2
          Batata  4
           Carne  0
Feijao_com_Carne  3
          Frango  0
         Iogurte  0
  Leite_Integral  2
            Ovos  2
 Torta_de_Cereja  0
;



### *Análise da solução mais diversificada*

#### **Pontos positivos:**
- O modelo selecionou mais alimentos do que antes.  
- Nenhum alimento está sendo consumido em excesso, respeitando o limite de **4 porções por alimento**.  

### **Pontos negativos:**
- Alguns alimentos ainda **não foram selecionados** (**Carne, Frango, Iogurte e Torta de Cereja**).  
- O consumo de **proteína** está concentrado principalmente no **Feijão com Carne e nos Ovos**, enquanto outras fontes, como **Carne e Frango**, não foram incluídas.
- A **diversidade alimentar aumentou**, mas isso afetou diretamente o custo da dieta — houve um acréscimo de **\$19.86** em relação ao valor ótimo (**\$54.54 → \$74.4**).


### **Tabela Comparativa Solução Ótima x Solução Diversificada**

| **Critério**                | **Solução Ótima** | **Solução Diversificada** |
|----------------------------|--------------------------------|--------------------------------|
| **Quantidade de Alimentos Únicos** | 5  | 6  |
| **Interações Simplex**    | 0  | 323  |
| **Iterações Barrier**     | 0  | 323  |
| **Custo Total**           | **54.54**  | **74.4**  |
| **Análise da Solução**    | Menor custo, menor diversidade  | Maior diversidade, custo maior  |


### *Legenda dos Critérios*

- **Quantidade de Alimentos Únicos**: Número total de alimentos distintos selecionados na solução final.
- **Interações Simplex**: Quantidade de iterações realizadas pelo método Simplex, um algoritmo usado para resolver problemas de otimização linear.
- **Iterações Barrier**: Número de iterações feitas pelo método Barrier, uma abordagem usada para resolver otimizações com restrições.
- **Custo Total**: Valor total da dieta considerando os preços dos alimentos escolhidos.
- **Análise da Solução**: Comparação qualitativa entre as duas soluções, destacando vantagens e desvantagens de cada abordagem.

### **Conclusão**

A escolha entre as duas soluções depende do objetivo: se a prioridade for minimizar custos, a Solução Ótima é preferível. Porém, se for desejável um cardápio mais variado e equilibrado, a Solução Diversificada pode ser a melhor opção, desde que o custo adicional seja aceitável. Pessoalmente, eu preferiria a *Solução Ótima*, pois é mais barata, tem menos custo operacional e diversidade alimentar é apenas um alimento a menos.